In [1]:
pip install transformers

In [2]:
pip install yfinance numpy

In [3]:
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from transformers import pipeline

# Initialize Hugging Face GPT-Neo model pipeline for LLM-powered agents
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B', device='cuda')

def llm_search_agent(instruction):
    """
    Uses GPT-Neo model to generate a response based on the given instruction.
    """
    response = generator(instruction, max_length=50, num_return_sequences=1)
    print("LLM Search Agent Response:", response[0]['generated_text'])
    # Here the LLM would ideally interpret and initiate a search for top-performing stocks
    return "Searching for top 10 performing stocks over the past 3 months..."

# Stock Performance Fetching Agent

def fetch_stock_performance(ticker):
    """
    Fetches the 3-month performance of a single stock ticker.
    Returns the ticker and its performance (percentage change).
    """
    try:
        stock_data = yf.Ticker(ticker)
        hist = stock_data.history(period="3mo")

        if not hist.empty:
            start_price = hist["Close"].iloc[0]
            end_price = hist["Close"].iloc[-1]
            percent_change = ((end_price - start_price) / start_price) * 100
            return ticker, percent_change
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
    return ticker, None

def fetch_top_10_stocks():
    """
    Fetches the top 10 performing stocks based on the last 3 months' performance.
    """
    tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "NVDA", "META", "NFLX", "ORCL", "INTC",
               "IBM", "AMD", "CRM", "BA", "DIS", "GE", "KO", "PEP", "COST", "MCD"]  # Extend as needed

    performance = {}

    # Use ThreadPoolExecutor to parallelize the data fetching process
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(fetch_stock_performance, ticker): ticker for ticker in tickers}

        for future in as_completed(futures):
            ticker, percent_change = future.result()
            if percent_change is not None:
                performance[ticker] = percent_change

    # Sort by performance and get the top 10 performing stocks
    top_10 = sorted(performance, key=performance.get, reverse=True)[:10]

    print("Top 10 Performing Stocks over the past 3 months:", top_10)
    return top_10


def data_fetcher(ticker, period="3mo"):
    try:
        stock_data = yf.Ticker(ticker)
        hist = stock_data.history(period=period)
        prices = hist["Close"].tolist()
        return prices
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def normalize_prices(prices):
    # Calculate normalized price (percent change)
    return [(prices[i] - prices[0]) / prices[0] * 100 for i in range(len(prices))]

def performance_analyzer(ticker_data):
    best_performer = None
    max_avg_return = -np.inf

    for ticker, normalized_prices in ticker_data.items():
        avg_return = sum(normalized_prices) / len(normalized_prices)
        print(f"{ticker}: Avg Return over 3 months = {avg_return}%")

        if avg_return > max_avg_return:
            max_avg_return = avg_return
            best_performer = ticker

    return best_performer, max_avg_return

# Main function to execute the multi-agent flow
def main():
    # Step 1: Use LLM agent to interpret the instruction
    instruction = "Find the top 10 performing stocks over the past 3 months and identify the best."
    llm_search_agent(instruction)

    # Step 2: Fetch top 10 stocks (this should ideally be dynamic)
    top_stocks = fetch_top_10_stocks()
    print("Top 10 Stocks Identified:", top_stocks)

    # Step 3: Fetch data and normalize prices for each stock
    ticker_data = {}
    for ticker in top_stocks:
        prices = data_fetcher(ticker)
        if prices:
            normalized_prices = normalize_prices(prices)
            ticker_data[ticker] = normalized_prices
            print(f"{ticker}: Normalized Prices = {normalized_prices}")

    # Step 4: Analyze performance and find the best-performing stock
    best_stock, best_return = performance_analyzer(ticker_data)
    print(f"Best Performing Stock: {best_stock} with an average return of {best_return}%")

if __name__ == "__main__":
    main()


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


LLM Search Agent Response: Find the top 10 performing stocks over the past 3 months and identify the best. They are ranked by which stock they made the biggest gains. Also include a description of the performance of the stock and the price increases for the stock over the last three months
Top 10 Performing Stocks over the past 3 months: ['TSLA', 'ORCL', 'NVDA', 'INTC', 'CRM', 'NFLX', 'AMZN', 'DIS', 'META', 'MCD']
Top 10 Stocks Identified: ['TSLA', 'ORCL', 'NVDA', 'INTC', 'CRM', 'NFLX', 'AMZN', 'DIS', 'META', 'MCD']
TSLA: Normalized Prices = [0.0, -1.2549972534179688, 3.9150009155273438, 0.69000244140625, 7.06999969482422, 8.05999755859375, 11.360000610351562, 10.550003051757812, 11.635002136230469, 5.3300018310546875, 10.160003662109375, 6.605003356933594, 4.605003356933594, 2.875, 3.1399993896484375, 7.05500030517578, 5.3000030517578125, 9.705001831054688, 15.084999084472656, 5.364997863769531, 8.135002136230469, 13.084999084472656, 14.06500244140625, 14.904998779296875, 15.144996643